In [2]:
! pip3 install pymysql
# ! pip install pymysql

import pymysql as MySQLdb
# import mysql.connector
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:,.8f}'.format

cnx = MySQLdb.connect(user='admin', password='rdx-mysql',
                              host='database-1.ccbb7nilplok.us-east-1.rds.amazonaws.com',
                              database='etl')

# cnx = mysql.connector.connect(user='admin', password='rdx-mysql',
#                               host='database-1.ccbb7nilplok.us-east-1.rds.amazonaws.com',
#                               database='etl')

    100% |████████████████████████████████| 51kB 13.0MB/s ta 0:00:01
You are using pip version 10.0.1, however version 20.1b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
def get_by_metric_name (cnx, name):
    cursor = cnx.cursor()
    query = ("SELECT RECON_ID,NEW_BUS_ID,SITE,MEASURED_VALUE FROM event_logs_3 WHERE METRIC_NAME = \"" 
             + name + "\"")    
    cursor.execute(query)
    values = list(cursor.fetchall())
    header = cursor.description
    cursor.close()
    columns = [i[0] for i in header]
    raw = pd.DataFrame(values, columns = columns)
    return raw

In [4]:
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

In [6]:
def group_agg (raw, level):
    group = raw.groupby(level)
    agg = group.MEASURED_VALUE.agg(['mean', 'count', 'std', percentile(10),percentile(20),percentile(30)
                                   ,percentile(40),percentile(50),percentile(60),percentile(70)
                                   ,percentile(80),percentile(90),percentile(99)])
    return agg

In [7]:
def extract_metric_group (raw):
    raw.loc[raw['SITE'] =='MMSB400MinutemanRd','SITE'] = 'MM400MinutemanRd'
    raw['MEASURED_VALUE'] = raw['MEASURED_VALUE'].astype(float)
    raw = raw[raw['MEASURED_VALUE'] <= 105]
    site_group_agg = group_agg(raw, 'SITE')
    cus_group_agg = group_agg(raw, 'NEW_BUS_ID')
    dev_group_agg = group_agg(raw, 'RECON_ID')
    return site_group_agg, cus_group_agg, dev_group_agg

In [8]:
def write_metric_group_to_csv (cnx, metric_name):
    raw = get_by_metric_name(cnx,metric_name)
    site_group_agg, cus_group_agg, dev_group_agg = extract_metric_group(raw)
    for file_name,df in zip([metric_name+'_site_group_agg', metric_name+'_cus_group_agg', metric_name+'_dev_group_agg']
                        ,[site_group_agg, cus_group_agg, dev_group_agg]):
        df.to_csv(file_name+'.csv')

In [9]:
write_metric_group_to_csv(cnx,'used')

In [10]:
write_metric_group_to_csv(cnx,'LDldFreeSpacePct')

In [ ]:
write_metric_group_to_csv(cnx,'utilization')

In [ ]:
write_metric_group_to_csv(cnx,'dskPercent')

In [ ]:
write_metric_group_to_csv(cnx,'PctFree')

In [ ]:
write_metric_group_to_csv(cnx,'PctTotal')